In [1]:
# Setup Reveal.JS
from traitlets.config.manager import BaseJSONConfigManager
from pathlib import Path
path = Path.home() / ".jupyter" / "nbconfig"
cm = BaseJSONConfigManager(config_dir=str(path))
# ipyleaflet hack to load full map in Reveal.js
# These settings are also injected into the notebook metadata
# (Edit -> Edit Notebook Metadata), which is the preferred method
cm.update(
    "rise",
    {"minScale": 1.25,
     "width": "80%",
     "transition": "none",
     #"start_slideshow_at": "beginning",
     #"auto_select": "none",
     #"autolaunch": "true"
    }
)

#import pdfkit

from ipywidgets.embed import embed_minimal_html

def save_ipyleaflet(m, filename):
    embed_minimal_html(filename + '.html', views=[m])
    
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(42)

In [2]:
from utils import AIUKSlides
bristol_center = (51.4545, -2.5879)

slides = AIUKSlides(local_center=bristol_center, isolines=[.0, .2, .4, .6, .8, 1.0],
                    width='600px', height='400px', grid_density=500)

# Some recent COVID-19 data

In [3]:
m = slides.map_covid_uk()
display(m)
save_ipyleaflet(m, 'covid_england')

Map(center=[53, -2], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_tex…

Case numbers going up/down in <span style="color:red">red</span>/<span style="color:blue">blue</span>.

# Zooming in on Bristol

In [4]:
m = slides.map_covid_local()
display(m)
save_ipyleaflet(m, 'covid_bristol')

Map(center=[51.4545, -2.5879], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

In [5]:
from utils import KDE
clf = KDE(bandwidth=0.005)

slides.train_local_classifier(clf)
slides.train_local_foreground()

# AI can distinguish between up/down areas

In [6]:
m = slides.map_local_classifier_foreground()
display(m)
save_ipyleaflet(m, 'covid_bristol_bc_kde')

Map(center=[51.4545, -2.5879], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

# What actually happens with discriminative models

In [ ]:
m = slides.map_local_classifier(fillopacity=0.4, lineopacity=1.0)
display(m)
save_ipyleaflet(m, 'covid_bristol_kde')

# Different model, similar issue

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf2 = RandomForestClassifier()
slides.train_local_classifier(clf2)
m = slides.map_local_classifier(fillopacity=0.4, lineopacity=1.0)
display(m)
save_ipyleaflet(m, 'covid_bristol_rf')

In the COVID-19 example we see that a discriminative model has no problem making confident -- but unjustified -- predictions in areas without training data. 

In [ ]:
display(slides.map_local_classifier(fillopacity=0.4, lineopacity=1.0))
save_ipyleaflet(m, 'covid_bristol_rf_2')

The issue is that a learned model usually operates without direct access to the data used to train it, and so has no way of knowing when it ventures out of its "comfort zone". 

Luckily there are techniques for identifying a model's "comfort zone": one such technique called [`Background Check`](https://reframe.github.io/background_check/) works by introducing an additional "background class" during training. 

This was in fact the technique used previously, showing only the "foreground" classes. 

In [ ]:
display(slides.map_local_classifier_foreground())
save_ipyleaflet(m, 'covid_bristol_rf_bc')

# Why does this matter?